<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/CHAPTER4%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%91%9C%ED%98%84%EA%B3%BC%20%ED%8A%B9%EC%84%B1%20%EA%B3%B5%ED%95%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 들어가며

2차원 실수형 배열을 가진 데이터로서 각 열이 샘플에 대해 설명하면 연속형 특성(continuous feature)이라고 가정했습니다. 하지만 많은 애플리케이션에서는 이렇게 데이터가 수집되지 않습니다. 일반적인 특성의 전형적인 형태는 숫자 값이 아닌 범주형 특성(categorical feature)(=이산형 특성(discrete feature))입니다.

연속형 특성과 범주형 특성은 회귀와 분류의 차이와 비슷합니다. 다만 출력이 아닌 입력에 대한 것이란 점에서 차이가 있습니다. 앞서 살폈던 연속형 특성의 예는 픽셀 밝기, 붓꽃 측정값입니다. 범주형 특성의 예는 제품 브랜드, 색상, 판매 분류(책, 옷, 하드웨어) 등입니다. 이런 특성들은 모두 상품을 묘사하는 속성이므로 연속된 값이 아닙니다. 제품과 제품 간에는 중간값이 없고 순서도 없습니다.

하지만 데이터의 특성이 어떤 형태를 취하는지보다 중요한 것은 데이터를 표현하는 방식입니다. 이 방식에 따라 머신러닝 모델 성능에 주는 영향이 큽니다. 데이터의 스케일을 조정하는 것이 매우 중요하다는 점을 강조했습니다. 예를 들면 측정치의 스케일을 조정하지 않았을 때 센티미터, 인치의 기준에 따라 큰 차이가 생깁니다. 스케일 조정뿐만 아니라 특성의 상호작용(특성 간의 곱)이나 일반적인 다항식을 추가 특성으로 넣는 것도 방법입니다.

특정 애플리케이션에 가장 적합한 데이터 표현을 찾는 것을 '특성 공학(feature engineering)'이라고 합니다. 올바른 데이터 표현을 찾는다면 지도 학습 모델에서 적절한 매개변수를 선택하는 것보다 모델 성능을 더 크게 제고할 수 있습니다.

# 4.1 범주형 변수

예제에 사용할 데이터는 [1994년 인구 조사 데이터베이스에서 추출한 미국 성인의 소득 데이터셋](https://github.com/rickiepark/introduction_to_ml_with_python/blob/master/data/adult.data)입니다. 이 데이터셋을 사용해서 특정 근로자의 수입이 50,000달러를 초과하는지 이하인지를 예측하고자 합니다.

In [10]:
pip install mglearn

In [11]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
%matplotlib inline

In [12]:
# 데이터셋 10개를 부르기 위한 코드입니다.
# 코드는 뒤에서 다시 설명합니다.
import os

data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'),
                   header=None, index_col=False,
                   names=['age', 'workclass', 'fnlwgt', 'education',
                          'education-num', 'marital-status', 'occupation',
                          'relationship', 'race', 'gender', 'capital-gain',
                          'capital-loss', 'hours-per-week', 'native-country',
                          'income'])
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
display(data.head(11))

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K
5,37,Private,Masters,Female,40,Exec-managerial,<=50K
6,49,Private,9th,Female,16,Other-service,<=50K
7,52,Self-emp-not-inc,HS-grad,Male,45,Exec-managerial,>50K
8,31,Private,Masters,Female,50,Prof-specialty,>50K
9,42,Private,Bachelors,Male,40,Exec-managerial,>50K


데이터셋에는 여러 특성이 있습니다. 근로자 나이(age), 고용형태(workclass), 교육수준(education), 성별(gender), 주당 근로시간(hours-per-week), 직업(occupation) 등입니다.

이 예측 작업은 소득(income)이 <=50k, >50k라는 두 클래스를 가진 분류 문제입니다. 정확한 소득을 예측하고자 한다면 회귀 문제입니다.

이 데이터셋에 있는 age와 hours-per-week는 연속형 특성입니다. 그리고 workclass, education, sex, occupation은 범주형 특성입니다. 범위가 아닌 고정된 목록 중에 하나를 값으로 가지며 정성적(qualitative)입니다.

먼저 로지스틱 회귀 분류기를 학습시킵니다. 공식을 다시 서술하면 아래와 같습니다. 

$\hat{y}=w[0] \times x[0] + w[1] \times x[1] + \cdots +w[p] \times x[p] + b > 0$

$w[i]$와 $b$는 훈련 세트로부터 학습되는 계수이고 $x[i]$는 입력 특성입니다. $x[i]$는 숫자여야 하므로 $x[2]$는 'masters'나 'Bachelors'가 될 수 없습니다. 따라서 로지스틱 회귀를 사용하려면 데이터를 다른 방식으로 표현해야 합니다.

## 4.1.1 원-핫-인코딩(가변수)

범주형 변수를 표현하는 데 가장 널리 쓰이는 방법은 '원-핫-인코딩(one-hot encoding)'입니다. '원-아웃-오브-엔 인코딩(one-out-of-N encoding)', '가변수(dummy variable)'로도 불립니다. 가변수는 범주형 변수를 0 또는 1 값을 가진 하나 이상의 새로운 특성으로 바꾼 것입니다. 이렇게 0과 1로 표현된 변수를 선형 이진 분류 공식에(모든 모델에) 적용할 수 있습니다. 

0과 1로 표현된 변수는 개수와 상관없이 범주마다 특성 하나로 표현합니다. 가령 'workclass' 특성에 클래스 네 가지가 존재한다면, 이 클래스 네 가지를 인코딩 하기 위해 새로운 특성 네 가지를 만듭니다. 어떤 사람의 workclass 값에 해당하는 특성은 1이고 나머지 세 특성은 0이 되는 식입니다. 다시 말하면 샘플마다 새로운 특성 하나만 1 값을 갖고 나머지 특성은 0입니다. 원핫, 원아웃오브엔이 의미하는 바입니다.

참고로 원-핫 인코딩은 통계학에서 사용하는 더미 코딩과 유사하지만 차이점이 존재합니다. 머신러닝에서는 특성의 범주를 각기 다른 이진 특성으로 바꾸지만, 통계학에서는 k개의 값을 가진 범주형 특성을 k-1개의 특성으로 변환하는 것이 일반적입니다. 데이터 행렬의 랭크 부족(rank deficient) 현상을 피할 목적입니다. 랭크 부족에 대해서 더 설명하자면, 열 랭크 부족(column rank deficient)은 범주 네 개를 특성 네 개로 인코딩 하면 맨 마지막 특성은 앞의 세 특성을 참조해서 예측할 수 있고, 한 열이 다른 열에 의존적이거나 열의 값이 모두 0인 경우를 뜻합니다. 행렬 분해 방식에 따라서 문제가 될 소지가 있으나 사이킷런에서는 이런 문제를 일으키지 않습니다.

판다스나 사이킷런을 사용해서 범주형 변수를 원-핫-인코딩으로 바꿀 수 있습니다. 먼저 판다스를 사용해서 csv 파일을 읽습니다.

In [13]:
# 이 파일은 열 이름을 나타내는 헤더가 없으므로 `header=None`으로 지정합니다.
# 열 이름을 지정하고자 `names` 매개변수에 리스트 형태의 값을 넣습니다.
data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'),
                   header=None, index_col=False,
                   names=['age', 'workclass', 'fnlwgt', 'education',
                          'education-num', 'marital-status', 'occupation',
                          'relationship', 'race', 'gender', 'capital-gain',
                          'capital-loss', 'hours-per-week', 'native-country',
                          'income'])
# 지정한 열의 값만 불러옵니다.
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
print(data)
display(data.head())

       age          workclass  ...          occupation  income
0       39          State-gov  ...        Adm-clerical   <=50K
1       50   Self-emp-not-inc  ...     Exec-managerial   <=50K
2       38            Private  ...   Handlers-cleaners   <=50K
3       53            Private  ...   Handlers-cleaners   <=50K
4       28            Private  ...      Prof-specialty   <=50K
...    ...                ...  ...                 ...     ...
32556   27            Private  ...        Tech-support   <=50K
32557   40            Private  ...   Machine-op-inspct    >50K
32558   58            Private  ...        Adm-clerical   <=50K
32559   22            Private  ...        Adm-clerical   <=50K
32560   52       Self-emp-inc  ...     Exec-managerial    >50K

[32561 rows x 7 columns]


,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


### 범주형 데이터 문자열 확인하기

데이터셋을 읽은 후에는 먼저 범주형 데이터를 유심히 살피는 것이 현명합니다. 정해진 범주를 벗어난 값이 있을 수도 있고, 철자나 대소문자가 틀린 값이 있을 수도 있으므로 반드시 데이터를 전처리해야 합니다. 예컨대 남성이 'man'으로 입력되지 않고 'male'로 입력됐다면 이 값들은 필시 같은 범주로 인식하도록 조치해야 합니다. 

열의 내용을 확인하려면 판다스에서 (데이터프레임(DataFrame)의 열을 나타내는)Series에 있는 `value_counts` 메서드를 사용합니다. 유일한 값이 몇 번씩 출력되는지 확인할 수 있습니다. 실제로는 모든 열의 값을 살펴야 하지만 여기에서는 'gender' 열의 값만 확인합니다.

In [14]:
print(data.gender.value_counts())

 Male      21790
 Female    10771
Name: gender, dtype: int64


두 가지 값만을 가지고 있습니다. 원-핫-인코딩으로 나타내기에 적합합니다.

판다스에서는 `get_dummies` 함수를 사용해서 문자열과 같은 객체 타입 특성이나 범주형(R의 factor와 유사한 판다스의 category를 의미합니다) 특성을 가진 열을 자동으로 변환합니다. 각 특성을 펼쳐서 생성하는 것입니다.

In [15]:
print('원본 특성:\n', list(data.columns), '\n')
data_dummies = pd.get_dummies(data)
print('get_dummies를 사용한 후의 특성:', list(data_dummies.columns))

원본 특성:
 ['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income'] 

get_dummies를 사용한 후의 특성: ['age', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'gender_ Female', 'gender_ Male', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct', 'occupati

In [16]:
display(data_dummies.head())

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,gender_ Female,gender_ Male,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,39,40,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,50,13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,38,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,53,40,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,28,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


연속형 특성인 age와 hours-per-week는 그대로지만 범주형 특성은 새로운 특성으로 확장되고 생성됐습니다.

`get_dummies` 함수의 `values` 속성을 사용해서 판다스 데이터프레임을 넘파이 배열로 바꿀 수 있습니다. 이렇게 바꾼 넘파이 배열로 머신러닝 모델을 학습시킬 수 있습니다.

위에서 서술했듯이 이 문제의 주제는 특정 근로자의 수입이 50,000달러를 초과하는지 아닌지를 예측하는 것입니다. 따라서 'income'을 타깃값으로 삼을 예정입니다. 먼저 모델을 학습시키기 전에 'income'으로 시작하는 두 열에 인코딩 된 타깃값을 분리해야 합니다. 출력값이나 출력값으로부터 유도된 변수를 특성 표현에 포함하는 것은 지도 학습 모델을 만들 때 저지르기 쉬운 실수입니다. 여하튼 특성을 포함한 열, 즉 'age'부터 'occupation_ Transport-moving'까지 모든 열을 추출합니다. 타깃값인 'income'을 뺀 모든 특성입니다.

참고로 이 예제에서는 타깃값도 원-핫-인코딩을 통해 0과 1로 변환했지만 사실 사이킷런에서는 문자열도 타깃값으로 사용할 수 있습니다. 'income' 열을 그대로 사용해도 괜찮습니다.

In [17]:
# 판다스에서 열 인덱싱은 넘파이와는 다르게 범위의 끝을 포함합니다.
# 'age' 열부터 'occupation_ Transport-moving' 열까지 포함하는 것입니다.
features = data_dummies.loc[:, 'age':'occupation_ Transport-moving']
print(features)
# 넘파이 배열을 추출합니다.
X = features.values
print(X)
# 타깃값으로 삼을 'income' 열을 y로 지정합니다.
y = data_dummies['income_ >50K'].values
print(y)
print('X.shape: {} y.shape: {}'.format(X.shape, y.shape))

       age  ...  occupation_ Transport-moving
0       39  ...                             0
1       50  ...                             0
2       38  ...                             0
3       53  ...                             0
4       28  ...                             0
...    ...  ...                           ...
32556   27  ...                             0
32557   40  ...                             0
32558   58  ...                             0
32559   22  ...                             0
32560   52  ...                             0

[32561 rows x 44 columns]
[[39 40  0 ...  0  0  0]
 [50 13  0 ...  0  0  0]
 [38 40  0 ...  0  0  0]
 ...
 [58 40  0 ...  0  0  0]
 [22 20  0 ...  0  0  0]
 [52 40  0 ...  0  0  0]]
[0 0 0 ... 0 0 1]
X.shape: (32561, 44) y.shape: (32561,)


사이킷런에서 사용할 수 있는 형태가 됐습니다. `LogisticRegression`을 적용해서 점수를 출력합니다.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
print('테스트 점수: {:.2f}'.format(logreg.score(X_test, y_test)))

테스트 점수: 0.81


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


**CAUTION**

본서에서 매우 중요한 내용을 CAUTION이라는 파트에서 설명하고 있습니다. 이 내용을 자세하게 서술합니다.

이 데이터 프레임에는 훈련 데이터와 테스트 데이터가 모두 담겨 있습니다. 훈련 세트와 테스트 세트를 `get_dummies` 함수를 사용해서 범주형 값을 같게 만들었습니다.

훈련 세트와 테스트 세트가 서로 다른 데이터 프레임 형태를 가지고 있다고 가정하겠습니다. 'workclass' 특성의 'Private Employee' 값이 테스트 세트에 없으면 판다스는 이 특성의 값이 세 개뿐이라고 판단하고 세 개의 가변수 특성을 만듭니다. 그러면 훈련 세트와 테스트 세트의 특성 개수가 달라집니다. 훈련 세트에서 학습된 모델을 테스트 세트에 적용할 수 없게 되는 것입니다.

더 심각하게 'workclass' 특성이 훈련 세트는 'Government Employee'와 'Private Employee'를 가지고, 테스트 세트는 'Self Employed'와 'Self Employed incorperated'를 가진다고 가정하면, 판다스는 양쪽 모두 두 개의 다른 새로운 가변수 특성을 추가하게 됩니다. 그렇게 만들어진 두 데이터 프레임의 특성 개수는 일단 같게 되지만, 가변수 특성 두 개는 훈련 세트와 테스트 세트에서 완전히 다른 의미를 지닙니다. 훈련 세트의 'Government Employee' 특성을 나타내는 열은 테스트 세트의 'Self Employed' 특성을 의미하는 것입니다. 이렇게 매우 다른 의미의 두 열을 단지 위치가 같다는 이유만으로 같은 것으로 여기기 때문에 이런 데이터로 모델을 만들면 결코 좋지 못한 결과를 얻습니다.

이를 방지하려면 위에서 제시한 예시 데이터처럼 훈련 데이터와 테스트 데이터 모두를 포함한 데이터 프레임으로 `get_dummies` 함수를 호출하거나, 각각 `get_dummies` 함수를 호출해서 훈련 세트와 테스트 세트의 열 이름을 비교해서 같은 속성인지 확인해야 합니다.

## 4.1.2 숫자로 표현된 범주형 특성

adult 데이터셋은 범주형 변수가 문자열로 인코딩 되어 있습니다. 철자에 오류가 있을 수 있지만 범주형 변수라는 점을 분명하게 나타냅니다. 하지만 저장 공간을 절약하거나 데이터 취합 방식에 따라 이 변수가 숫자로 인코딩 된 경우가 많습니다. 예를 들면 adult 데이터셋에 있는 인구조사 데이터를 설문지를 사용해서 수집했다고 가정한다면 'workclass'에 대한 질문의 대답은 값(8개)을 나타내는 체크 박스의 순서에 따라 0에서 8이 부여됩니다. 문자열인 'Private'가 숫자로 표현되는 것입니다. 이런 형태를 연속형인지 범주형인지 쉽게 알기는 어렵습니다. 다만 'workclass'의 값을 숫자(이산적)로 나타냈다는 사실을 인지한다면 연속형 변수로 다루지 않아야 한다는 점을 명확하게 깨달을 수 있습니다.

**CAUTION**

범주형 특성은 종종 숫자로 인코딩 되므로 무조건 연속형 특성으로 다뤄서는 안 됩니다. 숫자로 이루어진 특성이 연속형인지 이산형인지 원-핫-인코딩 된 것인지는 항상 명확하게 구분할 수 없습니다. 다만 인코딩 된 값 사이에 어떤 순서도 존재하지 않다면 이산형이라고 추론할 수 있습니다. 예를 들어 별 다섯 개가 만점인 평점 데이터의 경우라면 문제나 데이터, 어떤 머신러닝 알고리즘을 사용하느냐에 따라 적절한 인코딩 방법을 선택합니다. 물론 보통의 경우는 별의 개수에 따라 평점이 분류되므로 범주형으로 다루지만, 별점에 대한 평균을 구할 목적이라면 연속형으로 다룹니다. 영화 관람 등급의 경우처럼 순서를 가진 범주형 속성도 있습니다. 다양한 관점으로 데이터를 살펴볼 필요가 있습니다.

`get_dummies` 함수는 숫자 특성을 무조건 연속형으로 간주해서 가변수를 생성하지 않습니다. 이를 설명하기 위해 숫자 특성과 범주형 문자열 특성을 가진 데이터 프레임의 객체를 만듭니다.

In [21]:
demo_df = pd.DataFrame({'number_features': [0, 1, 2, 1],
                        'categorical_features': ['socks', 'fox', 'socks', 'box']})
display(demo_df)

,number_features,categorical_features
0,0,socks
1,1,fox
2,2,socks
3,1,box


'socks'는 숫자 특성이 동일하게 0이 아니라 각각 0과 2를 가졌습니다.

`get_dummies` 함수를 사용해서 결과를 확인합니다.

In [20]:
display(pd.get_dummies(demo_df))

,number features,categorical features_box,categorical features_fox,categorical features_socks
0,0,0,0,1
1,1,0,1,0
2,2,0,0,1
3,1,1,0,0


문자열 특성만 인코딩 되고 숫자 특성은 바뀌지 않았습니다.

만약 숫자로 된 특성도 가변수로 만든다면 `columns` 매개변수에 인코딩 하고자 하는 열을 명시합니다. 이렇게 하면 모든 특성을 범주형으로 간주합니다. 

In [25]:
demo_df['number_feature'] = demo_df['number_features'].astype(str)
display(pd.get_dummies(demo_df, columns=['number_features', 'categorical_features']))

,number_feature,number_features_0,number_features_1,number_features_2,categorical_features_box,categorical_features_fox,categorical_features_socks
0,0,1,0,0,0,0,1
1,1,0,1,0,0,1,0
2,2,0,0,1,0,0,1
3,1,0,1,0,1,0,0


물론 숫자를 문자열로 바꾸면(`demo_df['number_features'].astype(str)`) `columns` 매개변수에 값을 지정하지 않아도 가변수 특성이 만들어집니다. 위 코드에서 제시한 두 방법 중에 하나만 택해도 충분합니다.

# 4.2 OneHotEncoder와 ColumnTransformer: scikit-learn으로 범주형 변수 다루기

# 4.3 make_column_transformer로 간편하게 ColumnTransformer 만들기

# 4,4 구간 분할, 이산화 그리고 선형 모델, 트리 모델

# 4.5 상호작용과 다항식

# 4.6 일변량 비선형 변환

# 4.7 특성 자동 선택

## 4.7.1 일변량 통계

## 4.7.2 모델 기반 특성 선택

## 4.7.3 반복적 특성 선택

# 4.8 전문가 지식 활용

# 4.9 요약 및 정리